# Balance checks
This notebook calculates water and energy balances  for all six laugh tests.

## Expectations
Balance errors should be small, relative to maginutes of states and fluxes.

The Vanderborght case requires a special approach because its outputs are only calculated for a single time step. Therefore we need to calculate the balance using the initial conditions as start of the time window.

## Meta data

| Data  | Value  |
|:---|:---|
| Model name| Structure for Unifying Multiple Modelling Alternatives (SUMMA) |
| Model version  | See attributes in output .nc file |
| Model reference | Clark et al. (2015a,b) |
| Model runs by | R. Zolfaghari |
| Notebook code by | W. Knoben, A. Bennett |

In [1]:
# modules
from pathlib import Path
from operator import truediv 
import numpy as np
import pandas as pd
import xarray as xr # note, also needs netcdf4 library installed

In [2]:
# Specify the data locations relative to the notebook
file_paths = {'Celia'               : '../lt1_celia1990/output/celia1990_output_timestep.nc',
              'Miller clay'         : '../lt2_miller1998/output/millerClay_output_timestep.nc',
              'Miller loam'         : '../lt2_miller1998/output/millerLoam_output_timestep.nc',
              'Miller sand'         : '../lt2_miller1998/output/millerSand_output_timestep.nc',
              'Wigmosta exp = 1'    : '../lt4_wigmosta1994/output/syntheticHillslope-exp1_output_timestep.nc',
              'Wigmosta exp = 2'    : '../lt4_wigmosta1994/output/syntheticHillslope-exp2_output_timestep.nc',
              'Colbeck exp = 1'     : '../lt5_colbeck1976/output/colbeck1976-exp1_output_timestep.nc',
              'Colbeck exp = 2'     : '../lt5_colbeck1976/output/colbeck1976-exp2_output_timestep.nc',
              'Colbeck exp = 3'     : '../lt5_colbeck1976/output/colbeck1976-exp3_output_timestep.nc',
              'Mizoguchi'           : '../lt6_mizoguchi1990/output/mizoguchi1990_output_timestep.nc'}

In [3]:
# Selection parameters
# We can use these to remove the GRU and HRU dimensions from the output files > easier data handling
GRU, HRU = 0,0

In [4]:
# function to calculate the water balance components on a given time step
def calc_wb(dat,domain):
    
    # Input: xarray dataset with variables:
    # - time 
    # - nSoil
    # - nSnow
    # - nLayers
    # - mLayerDepth
    # - mLayerVolFracLiq
    # - mLayerVolFraqIce
    # - iLayerLiqFluxSoil
    # - mLayerTranspire
    # - mLayerBaseflow
    # - mLayerCompress
    
    # Make some storage variables
    vec_liqError  = []
    vec_state     = []
    vec_stateDiff = []
    
    # Find the timestep size [s]
    dt = round((dat['time'][1] - dat['time'][0]).values/np.timedelta64(1, 's'))
    print('    timestep size = ' + str(dt) + ' s.')

    # Intrinsic densities (needed for snow)
    iden_liq = 1000 # [kg m-3]
    iden_ice = 917
    
    # --- Water balance components soil ---
    # Start of time loop
    for t in range(0,len(dat['time'])-1):
       
        # Specify time as indices
        S_time = t 
        E_time = t+1
    
        # Get the layer variables at t=t
        S_nSoil   = dat['nSoil'].isel(time=S_time).values.astype('int')
        S_nSnow   = dat['nSnow'].isel(time=S_time).values.astype('int')
        S_nLayers = dat['nLayers'].isel(time=S_time).values.astype('int')
        
        # Do computations based on the specified domain
        if domain == 'snow':
            
            # Snow layer depths
            S_mLayerDepth = dat['mLayerDepth'].isel(time=S_time,midToto=slice(0,S_nSnow)).values # needs to start at index 0, not 1 in Python
            
            # Snow water at the start of t=t
            S_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=S_time,midToto=slice(0,S_nSnow)).values
            S_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=S_time,midToto=slice(0,S_nSnow)).values
            mass0 = sum( (S_mLayerVolFraqLiq * iden_liq + S_mLayerVolFraqIce * iden_ice) * S_mLayerDepth )
            snowBalance0 = mass0 / iden_liq

            # Layer variables at t=t+1
            E_nSoil   = dat['nSoil'].isel(time=E_time).values.astype('int')
            E_nSnow   = dat['nSnow'].isel(time=E_time).values.astype('int')
            E_nLayers = dat['nLayers'].isel(time=E_time).values.astype('int')
            E_mLayerDepth = dat['mLayerDepth'].isel(time=E_time,midToto=slice(0,E_nSnow)).values # needs to start at index 0, not 1 in Python

            # Rainfall flux between t=t and t=t+1
            scalarRainfall = (dat['scalarRainfall'].isel(time=E_time).values / iden_liq) * dt # [kg m-2 s-1] / [kg m-3] * [s] = [m]
            
            # Snowfall flux between t=t and t=t+1
            scalarSnowfall = (dat['scalarSnowfall'].isel(time=E_time).values / iden_ice) * dt # [kg m-2 s-1] / [kg m-3] * [s] = [m]
            
            # Melt flux between t=t and t=t+1
            scalarRainPlusMelt = dat['scalarRainPlusMelt'].isel(time=E_time).values * dt # [m s-1] * [s] = [m]
            
            # Snow water at the end of t=t; i.e. at the start of t=t+1
            E_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=E_time,midToto=slice(0,S_nSnow)).values
            E_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=E_time,midToto=slice(0,S_nSnow)).values
            mass1 = sum( (E_mLayerVolFraqLiq * iden_liq + E_mLayerVolFraqIce * iden_ice) * E_mLayerDepth )
            snowBalance1 = mass1 / iden_liq
            
            # Water balance error
            liqError = snowBalance1 - (snowBalance0 + scalarRainfall + scalarSnowfall - scalarRainPlusMelt)
            
            # Append
            vec_liqError.append(liqError)
            vec_state.append(snowBalance1)
            vec_stateDiff.append(snowBalance1 - snowBalance0)
        
        elif domain == 'soil':
        
            # Soil layer depths
            S_mLayerDepth = dat['mLayerDepth'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values # needs to start at index 0, not 1 in Python
        
            # Soil water at the start of t=t
            S_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values
            S_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values
            soilBalance0 = sum( (S_mLayerVolFraqLiq + S_mLayerVolFraqIce) * S_mLayerDepth )   
    
            # Layer variables at t=t+1
            E_nSoil   = dat['nSoil'].isel(time=E_time).values.astype('int')
            E_nSnow   = dat['nSnow'].isel(time=E_time).values.astype('int')
            E_nLayers = dat['nLayers'].isel(time=E_time).values.astype('int')
            E_mLayerDepth = dat['mLayerDepth'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values # needs to start at index 0, not 1 in Python
        
            # Vertical downward flux between t=t and t=t+1 (needs t=t+1)
            iLayerLiqFluxSoil_top = dat['iLayerLiqFluxSoil'].isel(time=E_time,ifcSoil=E_nSoil).values
            iLayerLiqFluxSoil_bot = dat['iLayerLiqFluxSoil'].isel(time=E_time,ifcSoil=0).values
            vertFlux = -1* (iLayerLiqFluxSoil_top - iLayerLiqFluxSoil_bot) * dt
    
            # Transpiration between t=t and t=t+1 (needs t=t+1)
            tranSink = dat['mLayerTranspire'].isel(time=E_time).sum().values*dt 
    
            # Baseflow between t=t and t=t+1 (needs t=t+1)
            baseSink = dat['mLayerBaseflow'].isel(time=E_time).sum().values*dt
    
            # Compression between t=t and t=t+1 (needs t=t+1)
            mLayerCompress = dat['mLayerCompress'].isel(time=E_time,midSoil=slice(0,E_nSoil)).values
            compSink = sum(mLayerCompress * E_mLayerDepth)
    
            # Soil water at the end of t=t; i.e. at the start of t=t+1
            E_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values
            E_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values
            soilBalance1 = sum( (E_mLayerVolFraqLiq + E_mLayerVolFraqIce) * E_mLayerDepth )
    
            # Water balance error
            liqError = soilBalance1 - (soilBalance0 + vertFlux + tranSink - baseSink - compSink)
        
            # Append
            vec_liqError.append(liqError)
            vec_state.append(soilBalance1)
            vec_stateDiff.append(soilBalance1 - soilBalance0)
            
    return vec_liqError, vec_state, vec_stateDiff

Processing starts here

In [5]:
# initiate some lists
test_name   = []
maxAbsErr   = []
meanAbsErr  = []
cumAbsErr   = []
maxRelErr1  = []
meanRelErr1 = []
maxRelErr2  = []
meanRelErr2 = []

# loop over the files
for test,file in file_paths.items():
    
    # progress
    print('Working on ' + test)
    
    # load the data
    if 'Colbeck' in test:
        dat = xr.open_dataset( file ).isel(hru=HRU).load() # Colbeck output has no GRU dimensions
        domain = 'snow'
    else:
        dat = xr.open_dataset( file ).isel(hru=HRU, gru=GRU).load()
        domain = 'soil'
    
    # Get water balance values
    wbe,states,dStates = calc_wb(dat,domain)

    # Calculate the relative error series
    rn = list(map(truediv, wbe, dStates))
    rrn = list(map(truediv, wbe, states)) # divide two lists element-wise
              
    # Remove NaNs if present
    rn  = [np.abs(val) for val in rn if str(val) != 'nan']
    rrn = [np.abs(val) for val in rrn if str(val) != 'nan']
        
    # Store the remaining metrics
    test_name.append(test)
    maxAbsErr.append(np.max(np.abs(wbe)))
    meanAbsErr.append(np.mean(np.abs(wbe)))
    cumAbsErr.append(np.sum(np.abs(wbe)))
    maxRelErr1.append(np.max(rn))
    meanRelErr1.append(np.mean(rn))      
    maxRelErr2.append(np.max(np.abs(rrn)))
    meanRelErr2.append(np.mean(np.abs(rrn)))

Working on Celia
    timestep size = 1800.0 s.
Working on Miller clay
    timestep size = 900.0 s.
Working on Miller loam
    timestep size = 900.0 s.
Working on Miller sand
    timestep size = 900.0 s.
Working on Wigmosta exp = 1
    timestep size = 3600.0 s.
Working on Wigmosta exp = 2
    timestep size = 3600.0 s.
Working on Colbeck exp = 1
    timestep size = 60.0 s.
Working on Colbeck exp = 2
    timestep size = 60.0 s.
Working on Colbeck exp = 3
    timestep size = 60.0 s.
Working on Mizoguchi
    timestep size = 60.0 s.


In [6]:
# Make a dataframe
results = pd.DataFrame( {'Test'               : test_name,
                         'Max abs err    [m]' : maxAbsErr,
                         'Mean abs err   [m]' : meanAbsErr,
                         'Cumm abs err   [m]' : cumAbsErr,
                         'Max rel err 1  [-]' : maxRelErr1,
                         'Mean rel err 1 [-]' : meanRelErr1,
                         'Max rel err 2  [-]' : maxRelErr2,
                         'Mean rel err 2 [-]' : meanRelErr2},
                       columns = ['Test', 'Max abs err    [m]', 'Mean abs err   [m]', 'Cumm abs err   [m]', \
                                  'Max rel err 1  [-]', 'Mean rel err 1 [-]', 'Max rel err 2  [-]', \
                                  'Mean rel err 2 [-]'])
results

,Test,Max abs err [m],Mean abs err [m],Cumm abs err [m],Max rel err 1 [-],Mean rel err 1 [-],Max rel err 2 [-],Mean rel err 2 [-]
0,Celia,2.775558e-16,6.122553e-17,7.285839e-15,7.250664e-13,1.344705e-13,2.086111e-15,5.118475e-16
1,Miller clay,1.185140e-06,1.008642e-07,2.400569e-05,5.687777e-04,1.618044e-04,1.725209e-06,1.378866e-07
2,Miller loam,1.065814e-14,1.153139e-15,2.744471e-13,3.564724e-12,3.986814e-13,8.574017e-15,8.254109e-16
3,Miller sand,1.130163e-11,3.546724e-13,8.441203e-11,5.537628e-09,1.957501e-10,3.882366e-12,1.546115e-13
4,Wigmosta exp = 1,2.164935e-15,8.619127e-17,8.670842e-14,2.708387e-08,2.373592e-10,1.121408e-14,4.504647e-16
5,Wigmosta exp = 2,4.607426e-15,1.127329e-16,1.134093e-13,1.060408e-09,1.320544e-11,1.844403e-14,4.761700e-16
6,Colbeck exp = 1,4.218847e-15,4.406603e-16,2.639555e-13,1.163348e-09,7.926049e-11,1.293560e-14,1.381681e-15
7,Colbeck exp = 2,3.330669e-15,2.423400e-16,1.451617e-13,2.983476e-09,7.184128e-11,8.893514e-15,6.595538e-16
8,Colbeck exp = 3,9.436896e-16,1.252940e-16,7.505108e-14,1.000000e+00,1.808696e-01,2.353089e-15,3.188003e-16
9,Mizoguchi,1.887379e-14,2.442013e-16,8.788803e-13,inf,inf,2.859665e-13,3.700019e-15


## Trial code

In [7]:
test = 'Vanderborght exp = 1'

In [8]:
file = '../lt3_vanderborght2005/output/vanderborght2005_exp1_output_timestep.nc'

In [9]:
file

'../lt3_vanderborght2005/output/vanderborght2005_exp1_output_timestep.nc'

In [10]:
dat = xr.open_dataset( file ).isel(hru=HRU, gru=GRU).load()

In [11]:
# Get ICs
if 'Vanderborght' in test:
    ICs = xr.open_dataset( '../lt3_vanderborght2005/initialConditions/summa_zInitialCond_vanderborght2005.nc' ).isel(hru=HRU)

FileNotFoundError: [Errno 2] No such file or directory: b'/home/stiff/plot_laughTest/lt3_vanderborght2005/initialConditions/summa_zInitialCond_vanderborght2005.nc'

In [ ]:
# Create datetime to assign to initial conditions
timeD = int( ICs['dt_init'].values[0] ) #np.int32 to int
timeE = dat['time'].values
timeS = timeE - np.timedelta64(timeD,'m')
timeS

In [ ]:
# Add a time dimension to the IC file
ICsn = ICs.expand_dims(time=timeS)

In [ ]:
# Merge data
    
# Variables with a 'midToto' dimension
var = ['mLayerDepth','mLayerVolFracLiq','mLayerVolFracIce']
vdb = xr.merge([dat[var].where(dat[var] != -9999, drop=True), \
                ICsn[var]])

# Variables that are not part of the initial conditions and can be merged easily
var = ['nSoil','nSnow','nLayers','iLayerLiqFluxSoil','mLayerTranspire','mLayerCompress','mLayerBaseflow']
for v in var:
    vdb[v] = dat[v]
    
# Fill the 'nLayers' variable on first time step
var = ['nSoil','nSnow']
for v in var:
    vdb[v].loc[dict(time=timeS)] = ICsn[v].isel(scalarv=0).values
vdb['nLayers'].loc[dict(time=timeS)] = vdb['nSoil'].loc[dict(time=timeS)].values + \
                                        vdb['nSnow'].loc[dict(time=timeS)]

In [ ]:
vdb

In [ ]:
wb = calc_wb(vdb,'soil')

In [ ]:
wb